# Inscribe AI: rectangles challenge

## Problem decomposition
1. Check intersection between two rectangles x and y.
2. Find all rectangles that intersect with rectangle x.
3. Find set of rectangles in which all members intersect each other, a complete graph.
4. Increase size of set until all other rectangles have been attempted to be added to set, a maximal set.
5. Attempt all combinations for each rectangle with all other rectangles to form largest (maximal) set.
6. If maximal set found compare to existing sets and store unique sets only.
7. Print maximal sets.

## Approach
- Represent as a graph where each rectangle is a vertex with intersections represented by edges.

In [1]:
from src.maximal_sets import FindMaximalSets
from src.rectangle import UUIDRectangle
from src.utils import PrintMaximalSets, CreateRectanglesFromData

In [2]:
# Recreate example
rectangle_1 = UUIDRectangle(1, 1, 5, 5, 1)
rectangle_2 = UUIDRectangle(4, 4, 3, 3, 2)
rectangle_3 = UUIDRectangle(6, 1, 1, 1, 3)
rectangle_4 = UUIDRectangle(2, 2, 3, 3, 4)
rectangle_5 = UUIDRectangle(1, 4, 1, 3, 5)
rectangles = [rectangle_1, rectangle_2, rectangle_3, rectangle_4, rectangle_5]

## Bron Kerbosch (BK) search
- Recursion with backtracking.
- Time complexity is exponential: `O(3^(N/3))`, `N` = number of vertices in graph.

In [3]:
# Solve example
max_sets_found = FindMaximalSets(rectangles)
print(*max_sets_found, sep = "\n")

{1, 2, 4}
{1, 5}
{3}


In [4]:
# A larger data sample
N = 30
rectangles = CreateRectanglesFromData()[0:N]
max_sets_found = FindMaximalSets(rectangles)
print(*sorted(max_sets_found, key=len, reverse=True), sep = "\n")

{0, 1, 3, 5, 8, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28}
{1, 3, 8, 12, 13, 16, 17, 20, 23, 24, 25, 27, 28}
{1, 2, 3, 6, 8, 11, 15, 16, 20, 23, 26, 29}
{2, 20, 6, 7, 26, 11, 29, 15}
{19, 7}
{4}
{9}
{10}


## Time complexity
- Handling exponential run time: can data structure be exploited?
- See [data exploration](data_exploration.ipynb)

## BK with pivot
- Reduce recursive calls to non-maximal sets by leveraging the fact that either the next vertex, `v` or one of it's non-neighbors will be part of the maximal set. We check if this is a maximal set prior to making the above decision.

In [5]:
max_sets_found = FindMaximalSets(rectangles, pivot=True)
print(*sorted(max_sets_found, key=len, reverse=True), sep = "\n")

{0, 1, 3, 5, 8, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28}
{1, 3, 8, 12, 13, 16, 17, 20, 23, 24, 25, 27, 28}
{1, 2, 3, 6, 8, 11, 15, 16, 20, 23, 26, 29}
{2, 20, 6, 7, 26, 11, 29, 15}
{19, 7}
{4}
{9}
{10}


## BK with vertex ordering by degree
- Select outer level search vertices based on degree, `d`, largest first. Minimizes size of graph passed to subsequent recursive calls.
- For sparse graphs run time: `O(d*N*3^(d/3))`, however the graph appears quite densely connected.

In [6]:
max_sets_found = FindMaximalSets(rectangles, by_degree=True)
print(*sorted(max_sets_found, key=len, reverse=True), sep = "\n")

{0, 1, 3, 5, 8, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28}
{1, 3, 8, 12, 13, 16, 17, 20, 23, 24, 25, 27, 28}
{1, 2, 3, 6, 8, 11, 15, 16, 20, 23, 26, 29}
{2, 20, 6, 7, 26, 11, 29, 15}
{19, 7}
{4}
{9}
{10}


## BK deployed with multiprocessing

In [11]:
# Increase sample size, limited to 100 processes
max_sets_found = FindMaximalSets(CreateRectanglesFromData()[0:100], multi_process=True)
print(*sorted(max_sets_found, key=len, reverse=True), sep = "\n")

{0, 1, 3, 5, 8, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 27, 28, 31, 32, 37, 38, 39, 42, 43, 45, 46, 50, 52, 53, 54, 55, 58, 60, 61, 64, 65, 66, 70, 71, 73, 75, 76, 78, 79, 80, 81, 85, 86, 88, 91, 92, 93, 94, 98}
{1, 3, 8, 12, 13, 16, 17, 20, 23, 24, 25, 27, 28, 31, 32, 37, 38, 39, 42, 43, 46, 50, 53, 54, 57, 58, 60, 61, 64, 65, 66, 70, 71, 73, 75, 76, 79, 80, 81, 83, 85, 88, 91, 92, 94, 98}
{1, 2, 3, 6, 8, 11, 15, 16, 20, 23, 26, 29, 30, 38, 39, 42, 43, 44, 50, 53, 54, 57, 60, 61, 65, 66, 67, 69, 75, 76, 81, 83, 85, 87, 91, 92, 94, 99}
{1, 3, 8, 13, 16, 20, 23, 24, 37, 38, 39, 42, 43, 44, 50, 53, 54, 57, 60, 61, 64, 65, 66, 70, 75, 76, 79, 81, 83, 85, 91, 92, 94, 98}
{2, 6, 11, 15, 20, 26, 29, 30, 35, 40, 42, 43, 44, 50, 53, 57, 60, 66, 67, 69, 74, 75, 76, 83, 85, 87, 91, 99}
{2, 6, 7, 11, 15, 26, 29, 30, 35, 36, 40, 41, 47, 49, 62, 67, 68, 69, 72, 74, 82, 87, 95, 96, 97, 99}
{2, 6, 7, 11, 15, 20, 26, 29, 30, 35, 40, 44, 47, 49, 57, 62, 67, 68, 69, 74, 83, 87, 95, 96, 99}
{96, 97, 

## Appendix
- Resources:
    - https://en.wikipedia.org/wiki/Bron%E2%80%93Kerbosch_algorithm
    - https://snap.stanford.edu/class/cs224w-readings/tomita06cliques.pdf